# RAG Evaluation with Bring Your Own Inference Responses (BYOI) on Amazon Bedrock

## Introduction

Amazon Bedrock RAG Evaluation capabilities now support "Bring Your Own Inference Responses" (BYOI), enabling you to assess any Retrieval-Augmented Generation system regardless of where it's deployed. This notebook demonstrates how to evaluate the quality of RAG systems using specialized metrics including the newly available citation metrics - Citation Precision and Citation Coverage - providing deep insights into how effectively your system uses retrieved information.

Through this guide, we'll explore:
- Setting up RAG evaluation configurations with BYOI
- The creation of retrieve-and-generate evaluation jobs
- Analyzing citation quality with the new precision and coverage metrics
- Monitoring evaluation progress 

## Prerequisites

Before we begin, make sure you have:
- An active AWS account with appropriate permissions
- Amazon Bedrock access enabled in your preferred region
- An S3 bucket for storing evaluation data and results
- An IAM role with necessary permissions for S3 and Bedrock
- RAG system outputs in the required BYOI format

> **Important**: The evaluation process requires access to Amazon Bedrock evaluator models. Make sure these are enabled in your account.

## Dataset Format for RAG BYOI

### Retrieve-and-Generate Evaluation Format
```json
{
  "conversationTurns": [
    {
      "prompt": {
        "content": [
          {
            "text": "Your prompt here"
          }
        ]
      },
      "referenceResponses": [
        {
          "content": [
            {
              "text": "Expected ground truth answer"
            }
          ]
        }
      ],
      "output": {
        "text": "Generated response text",
        "knowledgeBaseIdentifier": "third-party-RAG",
        "retrievedPassages": {
          "retrievalResults": [
            {
              "name": "Optional passage name",
              "content": {
                "text": "Retrieved passage content"
              },
              "metadata": {
                "source": "Optional metadata"
              }
            }
          ]
        },
        "citations": [
          {
            "generatedResponsePart": {
              "textResponsePart": {
                "span": {
                  "start": 0,
                  "end": 50
                },
                "text": "Part of the response that uses cited material"
              }
            },
            "retrievedReferences": [
              {
                "name": "Optional passage name",
                "content": {
                  "text": "Source passage for the citation"
                },
                "metadata": {
                  "source": "Optional metadata"
                }
              }
            ]
          }
        ]
      }
    }
  ]
}
```
## Implementation

First, let's set up our configuration parameters:

In [ ]:
#Upgrade Boto3
!pip install --upgrade boto3

In [2]:
# Verify boto3 installed successfully
import boto3
import json
import os
import sys
from datetime import datetime
import re
print(boto3.__version__)

1.38.33


In [3]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)
from utils import read_jsonl_to_dataframe, upload_training_data_to_s3

In [4]:
def split_user_message(user_msg):

    # Regex pattern to extract content between tags
    pattern = r"<context>(.*?)</context>\s*<question>(.*?)</question>"

    # Apply regex
    match = re.search(pattern, user_msg, re.DOTALL)

    if match:
        context = match.group(1).strip()
        question = match.group(2).strip()
        return context, question
        
    else:
        print("Pattern not found in the text")

In [5]:
def create_eval_record(results_row, labeled_row, model_identifier=None, kb_identifier='kb_id'):
    """
    Takes a batch inference result data set and builds an evaluation data set for use in bedrock evaluation
    
    Args:
        row:            pandas row
        max_records:    defaults to 1000 - max for bedrock evaluation
    
    Returns:
        dict: A formatted payload dictionary ready for Bedrock Evaluations API
    
    """
    result = results_row.to_dict()
    label = labeled_row.to_dict()

    retrieval_content, question = split_user_message(result['modelInput']['messages'][0]['content'][0]['text'])
    return {
        "conversationTurns": [
            {
                "prompt": {
                    "content": [
                        {
                            "text": question
                        }
                    ]
                },
                "referenceResponses": [
                    {
                        "content": label['modelInput']['messages'][1]['content'] # labeled answer content from assistant
                    }
                ],
                # "referenceContexts": [
                #     {
                #         "content": [
                #             {
                #                 "text": "A ground truth for a received passage"
                #             }
                #         ]
                #     }
                # ],
                "output": {
                    "text": result['modelOutput']['output']['message']['content'][0]['text'],
                    "modelIdentifier": model_identifier if model_identifier else 'placeholder_model',
                    "knowledgeBaseIdentifier": kb_identifier,
                    "retrievedPassages": {
                        "retrievalResults": [ # put context from squad example here
                            {
                                "name": f"retrieval_{results_row['recordId']}",
                                "content": {
                                    "text": retrieval_content
                                }
                            }
                        ]
                    }
                }
            }
        ]
    }

In [19]:
def create_evaluation_dataset(model_identifier, dataframe, labeled_df, rag_source_id=None, max_records=1000, output_filename=None):
    """
    Create an evaluation dataset from model results and write to a JSONL file.
    
    Parameters:
    -----------
    model_identifier : str
        Identifier for the model being evaluated
    dataframe : pandas.DataFrame
        DataFrame containing the results to evaluate
    labeled_df : pandas.DataFrame
        DataFrame containing labeled data with matching recordIds
    rag_source_id : str, optional
        Identifier for the knowledge base source
    max_records : int, optional
        Maximum number of records to process (default: 1000)
    output_filename : str, optional
        Custom filename for the output JSONL file
        
    Returns:
    --------
    str
        Path to the created JSONL file
    list
        The evaluation dataset as a list of dictionaries
    """
    import json
    
    eval_dataset = []
    kb_identifier = rag_source_id or f"{model_identifier}_kb"
    
    for ix, results_row in dataframe.head(max_records).iterrows():
        labeled_row = labeled_df[labeled_df['recordId'] == results_row['recordId']].iloc[0]
        eval_dataset.append(create_eval_record(
            results_row=results_row, 
            labeled_row=labeled_row,
            model_identifier=model_identifier, 
            kb_identifier=kb_identifier
        ))
    
    jsonl_file = output_filename or f"evaluation_data_{model_identifier}.jsonl"
    
    # Write all records to JSONL file
    with open(jsonl_file, 'w', encoding='utf-8') as f:
        for record in eval_dataset:
            f.write(json.dumps(record) + '\n')
    
    print(f"Successfully wrote {len(eval_dataset)} records to {jsonl_file}")
    return jsonl_file

In [26]:
def create_rag_evaluation_job(
    bedrock_client,
    model_identifier, 
    rag_source_id,
    input_data,
    role_arn,
    output_path,
    evaluator_model,
    metrics=None
):
    """
    Creates a Bedrock RAG evaluation job.

    Parameters:
    -----------
    bedrock_client : boto3.client
        The Bedrock client to use for creating the job
    model_identifier : str
        Identifier for the model being evaluated
    rag_source_id : str
        Identifier for the RAG knowledge base source
    input_data : str
        S3 URI for the input evaluation dataset
    role_arn : str
        IAM role ARN with sufficient permissions to run the evaluation
    output_path : str
        S3 URI where evaluation results will be stored
    evaluator_model : str
        Identifier for the model that will perform evaluations
    metrics : list, optional
        List of metrics to evaluate (defaults to standard RAG metrics)

    Returns:
    --------
    str
        The job ARN from the create_evaluation_job API call
    """
    from datetime import datetime
    
    # Set default RAG metrics if none provided
    if metrics is None:
        metrics = [
            "Builtin.Correctness",
            "Builtin.Completeness",
            "Builtin.Helpfulness",
            "Builtin.LogicalCoherence",
            "Builtin.Faithfulness",
            "Builtin.CitationPrecision",
            "Builtin.CitationCoverage"
        ]
    
    # Generate job name using timestamp for uniqueness
    retrieve_generate_job_name = f"citations-{model_identifier.replace('_','-')}-{datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}"
    
    # Create the evaluation job
    retrieve_generate_job = bedrock_client.create_evaluation_job(
        jobName=retrieve_generate_job_name,
        jobDescription="Evaluate retrieval and generation",
        roleArn=role_arn,
        applicationType="RagEvaluation",
        inferenceConfig={
            "ragConfigs": [
                {
                    "precomputedRagSourceConfig": {
                        "retrieveAndGenerateSourceConfig": {
                            "ragSourceIdentifier": rag_source_id
                        }
                    }
                }
            ]
        },
        outputDataConfig={
            "s3Uri": output_path
        },
        evaluationConfig={
            "automated": {
                "datasetMetricConfigs": [{
                    "taskType": "QuestionAndAnswer",  
                    "dataset": {
                        "name": f"{model_identifier}_dataset",
                        "datasetLocation": {
                            "s3Uri": input_data
                        }
                    },
                    "metricNames": metrics
                }],
                "evaluatorModelConfig": {
                    "bedrockEvaluatorModels": [{
                        "modelIdentifier": evaluator_model
                    }]
                }
            }
        }
    )
    
    return retrieve_generate_job['jobArn']

In [ ]:
labeled_df = read_jsonl_to_dataframe('labeled_data.jsonl')

evaluation_models = [
    {
        "model_name": "nova_premier",
        "batch_inference_results_file": "./batch_inference_results/premier_results.jsonl",
    },
    {
        "model_name": "nova_lite_distilled",
        "batch_inference_results_file": "./batch_inference_results/distillation_results.jsonl",
    }
]


## Create IAM Service Role
https://docs.aws.amazon.com/bedrock/latest/userguide/judge-service-roles.html


To use the Python SDK for creating an RAG evaluation job with your own inference responses, use the following steps. First, set up the required configurations, which should include your model identifier for the evaluator, IAM role with appropriate permissions, S3 paths for input data containing your inference responses, and output location for results.

## Create Evaluation Datasets and Submit Evaluation Jobs

In [ ]:
# Configure knowledge base and model settings
evaluator_model = 'us.anthropic.claude-3-5-sonnet-20241022-v2:0' # "<YOUR_EVALUATOR_MODEL>"
role_arn = "arn:aws:iam::228707323172:role/bedrock_eval_role" # "arn:aws:iam::<YOUR_ACCOUNT_ID>:role/<YOUR_IAM_ROLE>"
BUCKET_NAME = 'sample-data-us-east-1-228707323172-1' # Replace by your bucket name "<YOUR_S3_BUCKET_NAME>"
PREFIX = 'citations_distillation' # "<YOUR_BUCKET_PREFIX>"
# RAG_dataset_custom_name = "<YOUR_RAG_BYOI_DATASET_NAME>" # without the ".jsonl file extension

output_path = f"s3://{BUCKET_NAME}/{PREFIX}/"

# Create Bedrock client
bedrock_client = boto3.client('bedrock', region_name='us-east-1')

eval_details = []
for model in evaluation_models:
    dataframe = read_jsonl_to_dataframe(model['batch_inference_results_file'])
    eval_dataset_local_file = create_evaluation_dataset(model['model_name'],dataframe, labeled_df)
    eval_dataset_s3 = upload_training_data_to_s3(bucket_name=BUCKET_NAME, local_file_path=eval_dataset_local_file,prefix=PREFIX)
    eval_job_arn = create_rag_evaluation_job(
        bedrock_client,
        model_identifier=model['model_name'], 
        rag_source_id = f"{model['model_name']}_kb",
        input_data = eval_dataset_s3,
        role_arn=role_arn,
        output_path=output_path,
        evaluator_model=evaluator_model,
        metrics=None
    )
    eval_details.append({**model, 
                         "eval_dataset_local_file": eval_dataset_local_file, 
                         "eval_dataset_s3": eval_dataset_s3, 
                         "eval_job_arn": eval_job_arn, 
                         "evaluator_model": evaluator_model})


Uploading evaluation_data_nova_premier.jsonl to bucket sample-data-us-east-1-228707323172-1 with prefix citations_distillation...
Successfully uploaded evaluation_data_nova_premier.jsonl to S3 bucket!
File S3 URI: s3://sample-data-us-east-1-228707323172-1/citations_distillation/evaluation_data_nova_premier.jsonl
Uploading evaluation_data_nova_lite_distilled.jsonl to bucket sample-data-us-east-1-228707323172-1 with prefix citations_distillation...
Successfully uploaded evaluation_data_nova_lite_distilled.jsonl to S3 bucket!
File S3 URI: s3://sample-data-us-east-1-228707323172-1/citations_distillation/evaluation_data_nova_lite_distilled.jsonl


## Configuring a Retrieve and Generate RAG Evaluation Job with BYOI

The code below creates an evaluation job that analyzes both retrieval and generation quality from your RAG system. The most significant aspect is the `precomputedRagSourceConfig` parameter, which enables the Bring Your Own Inference capability. This configuration tells Bedrock to evaluate pre-generated responses rather than generating new ones.

Note how we're configuring a rich set of evaluation metrics, including the new citation metrics:

- **CitationPrecision**: Measures how accurately your RAG system cites sources by evaluating whether cited passages actually contain the information used in the response
- **CitationCoverage**: Evaluates how well the response's content is supported by its citations, focusing on whether all information derived from retrieved passages has been properly cited

The `ragSourceIdentifier` parameter must match the identifier in your dataset (in this example, "third-party-RAG"), creating the link between your evaluation configuration and the responses you've provided. The job will analyze your RAG system's performance across multiple dimensions, providing comprehensive insights into both information retrieval accuracy and generation quality.

In [ ]:
model_identifier = 'nova_premier'

nova_premier_eval_arn = create_rag_evaluation_job(
    bedrock_client,
    model_identifier, 
    rag_source_id = f"{model_identifier}_kb",
    input_data = nova_premier_dataset_s3,
    role_arn=role_arn,
    output_path=output_path,
    evaluator_model=evaluator_model,
    metrics=None
)

model_identifier = 'nova_lite_distilled'

nova_distilled_eval_arn = create_rag_evaluation_job(
    bedrock_client,
    model_identifier, 
    rag_source_id = f"{model_identifier}_kb",
    input_data = nova_lite_distilled_dataset_s3,
    role_arn=role_arn,
    output_path=output_path,
    evaluator_model=evaluator_model,
    metrics=None
)

## Monitoring Your RAG Evaluation Jobs

After submitting your evaluation jobs, you'll want to monitor their progress. The code below demonstrates how to check the status of both job types:

You can run this code periodically to track your job's progress through its lifecycle. Typical status values include "IN_PROGRESS", "COMPLETED", or "FAILED". Once a job reaches "COMPLETED" status, you can proceed to retrieve and analyze the evaluation results from the S3 output location you specified when creating the job.

In [ ]:
# Check status of retrieve-and-generate job
retrieve_generate_job_arn = retrieve_generate_job['jobArn']
retrieve_generate_status = bedrock_client.get_evaluation_job(jobIdentifier=retrieve_generate_job_arn)
print(f"Retrieve-and-Generate Job Status: {retrieve_generate_status['status']}")

## Conclusion

In this guide, we've explored how to leverage Amazon Bedrock RAG Evaluation capabilities with Bring Your Own Inference Responses to assess any RAG system's performance. Key advantages of this approach include:

- **Platform independence**: Evaluate RAG systems deployed anywhere - on Amazon Bedrock, other cloud providers, or on-premises
- **Comprehensive assessment**: Analyze both retrieve and generate quality with specialized metrics
- **Citation quality insights**: Leverage the new citation metrics to ensure responses are properly grounded in source information
- **Systematic benchmarking**: Compare different RAG implementations to make data-driven optimization decisions

By implementing regular evaluation workflows using these capabilities, you can continuously improve your RAG systems to deliver more accurate, relevant, and well-attributed responses. Whether you're fine-tuning retrieval strategies, optimizing prompt engineering, or exploring different foundation models for generation, these evaluation tools provide the quantitative insights needed to guide your development process.